In [1]:
import os
import h5py
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
os.environ['CUDA_VISIBLE_DEVICES']='0'

tf.enable_eager_execution()
print(tf.__version__)

/home/kjakkala/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kjakkala/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kjakkala/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kjakkala/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.

1.14.0


In [2]:
num_classes     = 14
batch_size      = 16
epochs          = 1000
learning_rate   = 0.0001
num_features    = 256
gen_activation  = 'selu'
notes           = "resnet_pretraining" 

log_data = "num_classes-{}_batch_size-{}_lr-{}_num_features-{}_gen_act-{}_notes-{}".format(num_classes,
                                                                                           batch_size,
                                                                                           learning_rate,
                                                                                           num_features,
                                                                                           gen_activation,
                                                                                           notes)

In [3]:
#Read data
hf = h5py.File('/home/kjakkala/mmwave/data/original_source_data.h5', 'r')
X_data = np.expand_dims(hf.get('X_data'), axis=-1).astype(np.float32)
y_data = np.array(hf.get('y_data')).astype(np.int32)[:, 0]
classes = list(hf.get('classes'))
classes = [n.decode("ascii", "ignore") for n in classes]
hf.close()
print(X_data.shape, y_data.shape, "\n", classes, len(classes))

#remove data for people in new dataset
del_labels = ["bivor", "champ", "kalvik", "prabhu", "pu_wang", "wei"]

for iden in del_labels:
  X_data = np.delete(X_data, np.where(y_data == classes.index(iden))[0], 0)
  y_data = np.delete(y_data, np.where(y_data == classes.index(iden))[0], 0)
  y_data[y_data >= classes.index(iden)] -= 1
  del classes[classes.index(iden)]

print(X_data.shape, y_data.shape, "\n", classes, len(classes))

(1976, 128, 1024, 1) (1976,) 
 ['amit_pandit', 'archit', 'bivor', 'champ', 'dan_han', 'dinesh', 'donglin', 'giang_dao', 'haoxin_wang', 'kalvik', 'kayan', 'minwoo_lee', 'prabhu', 'pu_wang', 'shamika', 'siqi_huang', 'sneha', 'srisanmathi', 'ting', 'wei'] 20
(1387, 128, 1024, 1) (1387,) 
 ['amit_pandit', 'archit', 'dan_han', 'dinesh', 'donglin', 'giang_dao', 'haoxin_wang', 'kayan', 'minwoo_lee', 'shamika', 'siqi_huang', 'sneha', 'srisanmathi', 'ting'] 14


In [4]:
#split data into train and test
y_data = (np.eye(len(classes))[y_data]).astype(np.int32)
X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                    y_data,
                                                    stratify=y_data,
                                                    test_size=0.10,
                                                    random_state=42)

print(X_train.shape, y_train.shape,  X_test.shape, y_test.shape)

(1248, 128, 1024, 1) (1248, 14) (139, 128, 1024, 1) (139, 14)


In [5]:
#standardise dataset
src_mean = np.mean(X_train)
X_train -= src_mean
src_std  = np.std(X_train)
X_train /= src_std
X_test -= src_mean
X_test /= src_std

print(X_train.shape, y_train.shape,  X_test.shape, y_test.shape)

(1248, 128, 1024, 1) (1248, 14) (139, 128, 1024, 1) (139, 14)


In [6]:
#get tf.data objects for each set
src_train_set = tf.data.Dataset.from_tensor_slices((X_train, y_train))
src_train_set = src_train_set.shuffle(X_train.shape[0])
src_train_set = src_train_set.batch(batch_size, drop_remainder=True)
src_train_set = src_train_set.prefetch(tf.data.experimental.AUTOTUNE)

src_test_set = tf.data.Dataset.from_tensor_slices((X_test, y_test))
src_test_set = src_test_set.batch(batch_size, drop_remainder=False)
src_test_set = src_test_set.prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
L2_WEIGHT_DECAY = 1e-4
BATCH_NORM_DECAY = 0.9
BATCH_NORM_EPSILON = 1e-5

class GaussianNoise(tf.keras.layers.Layer):
  def __init__(self, std):
    super(GaussianNoise, self).__init__()
    self.std = std

  def build(self, input_shapes):
    pass

  def call(self, inputs, training=False):
    eps = tf.random.normal(shape=tf.shape(inputs), mean=0.0, stddev=self.std)
    return tf.where(training, inputs + eps, inputs)

class IdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters, stage, block, activation='relu'):
    self.activation = activation
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    super().__init__(name='stage-' + str(stage) + '_block-' + block)

    filters1, filters2, filters3 = filters
    bn_axis = -1

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1),
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2a')
    self.bn2a = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2a')

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size,
                                         padding='same',
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2b')
    self.bn2b = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2b')

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1),
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2c')
    self.bn2c = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2c')

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.keras.layers.Activation(self.activation)(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.keras.layers.Activation(self.activation)(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x = tf.keras.layers.add([x, input_tensor])
    x = tf.keras.layers.Activation(self.activation)(x)
    return x


"""A block that has a conv layer at shortcut.

Note that from stage 3,
the second conv layer at main path is with strides=(2, 2)
And the shortcut should have strides=(2, 2) as well

Args:
  kernel_size: the kernel size of middle conv layer at main path
  filters: list of integers, the filters of 3 conv layer at main path
  stage: integer, current stage label, used for generating layer names
  block: 'a','b'..., current block label, used for generating layer names
  strides: Strides for the second conv layer in the block.

Returns:
  A Keras model instance for the block.
"""
class ConvBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters, stage, block, strides=(2, 2), activation='relu'):
    self.activation = activation
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    super().__init__(name='stage-' + str(stage) + '_block-' + block)

    filters1, filters2, filters3 = filters
    bn_axis = -1

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1),
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2a')
    self.bn2a = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2a')

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size,
                                         strides=strides,
                                         padding='same',
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2b')
    self.bn2b = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2b')

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1),
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '2c')
    self.bn2c = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '2c')

    self.conv2s = tf.keras.layers.Conv2D(filters3, (1, 1),
                                         strides=strides,
                                         use_bias=False,
                                         kernel_initializer='he_normal',
                                         kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                         name=conv_name_base + '1')
    self.bn2s = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                   momentum=BATCH_NORM_DECAY,
                                                   epsilon=BATCH_NORM_EPSILON,
                                                   name=bn_name_base + '1')
    self.gauss1   = GaussianNoise(1)

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.keras.layers.Activation(self.activation)(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.keras.layers.Activation(self.activation)(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    shortcut = self.conv2s(input_tensor)
    shortcut = self.bn2s(shortcut, training=training)

    x = tf.keras.layers.add([x, shortcut])
    x = tf.keras.layers.Activation(self.activation)(x)
    x = self.gauss1(x)
    return x


"""Instantiates the ResNet50 architecture.

Args:
  num_classes: `int` number of classes for image classification.

Returns:
    A Keras model instance.
"""
class ResNet50(tf.keras.Model):
  def __init__(self, num_classes, num_features, activation='relu'):
    super().__init__(name='generator')
    bn_axis = -1
    self.activation = activation

    self.conv1 = tf.keras.layers.Conv2D(32, (7, 7),
                                        strides=(2, 2),
                                        padding='valid',
                                        use_bias=False,
                                        kernel_initializer='he_normal',
                                        kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                        name='conv1')
    self.bn1 = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                                  momentum=BATCH_NORM_DECAY,
                                                  epsilon=BATCH_NORM_EPSILON,
                                                  name='bn_conv1')
    self.act1 = tf.keras.layers.Activation(self.activation, name=self.activation+'1')
    self.max_pool1 = tf.keras.layers.MaxPooling2D((3, 3),
                                                  strides=(2, 2),
                                                  padding='same',
                                                  name='max_pool1')

    self.blocks = []
    self.blocks.append(ConvBlock(3, [32, 32, 128], strides=(1, 1), stage=2, block='a', activation=self.activation))
    self.blocks.append(IdentityBlock(3, [32, 32, 128], stage=2, block='b', activation=self.activation))

    self.blocks.append(ConvBlock(3, [64, 64, 256], stage=3, block='a', activation=self.activation))
    self.blocks.append(IdentityBlock(3, [64, 64, 256], stage=3, block='b', activation=self.activation))

    self.blocks.append(ConvBlock(3, [64, 64, 256], stage=4, block='a', activation=self.activation))
    self.blocks.append(IdentityBlock(3, [64, 64, 256], stage=4, block='b', activation=self.activation))

    self.avg_pool = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool')
    self.fc1 = tf.keras.layers.Dense(num_features,
                                     activation=self.activation,
                                     kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                     kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                     bias_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                     name='fc1')
    self.logits = tf.keras.layers.Dense(num_classes,
                                        activation=None,
                                        kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
                                        kernel_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                        bias_regularizer=tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                        name='logits')

  def call(self, img_input, training=False):
    x = self.conv1(img_input)
    x = self.bn1(x, training=training)
    x = self.act1(x)
    x = self.max_pool1(x)

    for block in self.blocks:
      x = block(x)

    x = self.avg_pool(x)
    fc1 = self.fc1(x)
    logits = self.logits(fc1)
    return logits, fc1

In [8]:
def get_cross_entropy_loss(labels, logits):
  loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logits)
  return tf.reduce_mean(loss)

def virtual_adversarial_images(images, logits, pert_norm_radius=3.5):  
  with tf.GradientTape() as tape:
    # Get normalised noise matrix
    noise = tf.random.normal(shape=tf.shape(images))
    noise = 1e-6 * tf.nn.l2_normalize(noise, axis=tf.range(1, len(noise.shape)))

    # Add noise to image and get new logits
    noise_logits, _ = generator(images + noise, 
                                tf.constant(False, dtype=tf.bool))

    # Get loss from noisey logits
    noise_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=logits, logits=noise_logits)
    noise_loss = tf.reduce_mean(noise_loss)

  # Based on perturbed image loss, get direction of greatest error
  adversarial_noise = tape.gradient(noise_loss, 
                                    [noise],
                                    unconnected_gradients='zero')[0]

  adversarial_noise = tf.nn.l2_normalize(adversarial_noise, 
                                         axis=tf.range(1, 4))

  # return images with adversarial perturbation
  return images + pert_norm_radius * adversarial_noise

def mixup_preprocess(x, y, batch_size, alpha=1):
    # random sample the lambda value from beta distribution.
    weight     = np.random.beta(alpha, alpha, batch_size)
    x_weight   = weight.reshape(batch_size, 1, 1, 1)
    y_weight   = weight.reshape(batch_size, 1)
    
    # Perform the mixup.
    indices = tf.random.shuffle(tf.range(batch_size))
    mixup_images = (x * x_weight) + (tf.gather(x, indices) * (1 - x_weight))
    mixup_labels = (y * y_weight) + (tf.gather(y, indices) * (1 - y_weight))    
    
    return mixup_images, tf.nn.softmax(mixup_labels)

In [9]:
train_total_loss         = tf.keras.metrics.Mean(name='train_total_loss')
train_src_vat_loss       = tf.keras.metrics.Mean(name='train_src_vat_loss')
train_src_mixup_loss     = tf.keras.metrics.Mean(name='train_src_mixup_loss')
train_cross_entropy_loss = tf.keras.metrics.Mean(name='train_cross_entropy_loss')
src_test_accuracy        = tf.keras.metrics.CategoricalAccuracy(name='src_test_accuracy')
src_train_accuracy       = tf.keras.metrics.CategoricalAccuracy(name='src_train_accuracy')

@tf.function
def train_gen_step(src_images, src_labels):  
  with tf.GradientTape() as gen_tape:
    #Logits
    src_logits, src_enc = generator(src_images, training=True)
    
    #VAT
    src_adver_images    = virtual_adversarial_images(src_images, tf.nn.softmax(src_logits))
    src_adver_logits, _ = generator(tf.stop_gradient(src_adver_images), training=True)
    
    #MixUp
    src_mixup_images, src_mixup_labels = mixup_preprocess(src_images, src_logits, batch_size)
    src_mixup_logits, _                = generator(tf.stop_gradient(src_mixup_images),
                                                   training=True)
    
    cross_entropy_loss  = get_cross_entropy_loss(labels=src_labels, 
                                                 logits=src_logits)
    src_vat_loss        = get_cross_entropy_loss(labels=tf.nn.softmax(tf.stop_gradient(src_logits)),
                                                 logits=src_adver_logits)
    src_mixup_loss      = get_cross_entropy_loss(labels=tf.stop_gradient(src_mixup_labels), 
                                                 logits=src_mixup_logits)

    total_loss = cross_entropy_loss + \
                 1    * src_mixup_loss +\
                 1    * src_vat_loss
    
  gen_gradients = gen_tape.gradient(total_loss, generator.trainable_variables)
  gen_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))

  src_train_accuracy(src_labels, src_logits)
  train_cross_entropy_loss(cross_entropy_loss)
  train_src_mixup_loss(src_mixup_loss)
  train_src_vat_loss(src_vat_loss)
  train_total_loss(total_loss)

@tf.function
def test_source_step(source_images, source_labels):
  source_logits, _ = generator(source_images, training=False)
  src_test_accuracy(source_labels, source_logits)

In [10]:
train_template = 'Epoch: {:03d}, TotalL: {:.4f}, CrossE: {:.4f}, Src VAT: {:.4f}, Src MixUp: {:.4f}, Src Train Acc: {:.2f}, '
test_template  = 'Src Test Acc: {:.2f}'

generator      = ResNet50(num_classes, num_features, gen_activation)
gen_optimizer  = tf.keras.optimizers.Adam(learning_rate = learning_rate, beta_1 = 0.5)

summary_writer = tf.contrib.summary.create_file_writer('../logs/{}'.format(log_data), flush_millis=10000)
summary_writer.set_as_default()
global_step = tf.train.get_or_create_global_step()

def log_loss():
  with tf.contrib.summary.always_record_summaries():
    tf.contrib.summary.scalar("train_total_loss", train_total_loss.result())
    tf.contrib.summary.scalar("train_cross_entropy_loss", train_cross_entropy_loss.result())
    tf.contrib.summary.scalar("src_train_accuracy", src_train_accuracy.result())
    tf.contrib.summary.scalar("train_src_vat_loss", train_src_vat_loss.result())
    tf.contrib.summary.scalar("train_src_mixup_loss", train_src_mixup_loss.result())
    tf.contrib.summary.scalar("src_test_accuracy", src_test_accuracy.result())
    
for epoch in range(epochs):
  global_step.assign_add(1)  
    
  for source_data in src_train_set:
    train_gen_step(source_data[0], source_data[1])    

  print(train_template.format(epoch+1,
                              train_total_loss.result(),
                              train_cross_entropy_loss.result(),
                              train_src_vat_loss.result(),
                              train_src_mixup_loss.result(),
                              src_train_accuracy.result()*100), end="")

  for source_data in src_test_set:
    test_source_step(source_data[0], source_data[1])
    
  print(test_template.format(src_test_accuracy.result()*100))

  if (epoch % 100 == 0):
    generator.save_weights('../checkpoints/resnet_pretrained_epoch-{}_test_accuracy-{:.2f}'.format(epoch, src_test_accuracy.result()*100))
  
  log_loss()
  
  train_total_loss.reset_states()
  train_cross_entropy_loss.reset_states()
  src_train_accuracy.reset_states()
  train_src_vat_loss.reset_states()
  train_src_mixup_loss.reset_states()
  src_test_accuracy.reset_states()
  

W0828 19:09:50.388742 139875714737920 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0828 19:09:51.846623 139875714737920 deprecation.py:323] From /home/kjakkala/.local/lib/python3.5/site-packages/tensorflow/python/keras/backend.py:4075: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 001, TotalL: 7.8679, CrossE: 2.6553, Src VAT: 2.6059, Src MixUp: 2.6068, Src Train Acc: 8.49, Src Test Acc: 7.19
Epoch: 002, TotalL: 7.7121, CrossE: 2.5531, Src VAT: 2.5733, Src MixUp: 2.5857, Src Train Acc: 10.02, Src Test Acc: 12.23
Epoch: 003, TotalL: 7.1629, CrossE: 2.3031, Src VAT: 2.4049, Src MixUp: 2.4549, Src Train Acc: 16.91, Src Test Acc: 28.06
Epoch: 004, TotalL: 6.5726, CrossE: 2.0605, Src VAT: 2.2104, Src MixUp: 2.3017, Src Train Acc: 25.40, Src Test Acc: 30.22
Epoch: 005, TotalL: 6.0704, CrossE: 1.8537, Src VAT: 2.0442, Src MixUp: 2.1725, Src Train Acc: 34.13, Src Test Acc: 40.29
Epoch: 006, TotalL: 5.6402, CrossE: 1.6874, Src VAT: 1.8917, Src MixUp: 2.0611, Src Train Acc: 41.19, Src Test Acc: 41.73
Epoch: 007, TotalL: 5.2774, CrossE: 1.5410, Src VAT: 1.7766, Src MixUp: 1.9598, Src Train Acc: 47.36, Src Test Acc: 43.17
Epoch: 008, TotalL: 4.9489, CrossE: 1.4207, Src VAT: 1.6627, Src MixUp: 1.8656, Src Train Acc: 51.92, Src Test Acc: 51.80
Epoch: 009, TotalL: 4.7264